# API example

## Creating a game

Creating a game and loading a scenario from a YAML file.

In [ ]:
from warshard.game import Game

game = Game(
    headless = True # If false, another thread is launched which displays the gamestate using pygame
    log_file_path
)

## Running a game

There are two main ways to run a game : either by running entire turns at once, or individual phases of the turns.

The main interface is the Orders list.

You create a list of Orders, like this, and then pass them to the turn functions. The game will then attempt to execute the orders.


### Order list

Explain what an order is, what a putative order is

- You can and should give multiple orders fo the same units to be executed in sequence for the movement. For example : if you want to move unit 6 from hex 1,2 to hex 1,5, you need to write three move orders IN THE CORRECT ORDER FOR THE SAME UNIT : "6 move to 1,3" then "6 move to 1,4" then "6 move to 1,5" and those must be IN THE CORRECT ORDER in the pending_orders list, since we execute orders in a FIFO fashion

- Putative orders are orders that will only be executed when checking for retreats or advances (after a fight in both case.) Putative orders cover both advance and retreats, and are also FIFO

- Orders do not need to be differentiated by type : for instance, an Order will be interpreted a movement if the target hex is empty, or as combat if it contains an enemy.

In [ ]:
# In practice, you can set up a google sheet for players to input orders,
# read it in csv automatically every X minute, and turn that into order lists

import pandas as pd
from warshard.actions import Order

# Replace with the appropriate URL to your Google Sheets CSV
google_sheet_url = "https://docs.google.com/spreadsheets/d/your-sheet-id/gviz/tq?tqx=out:csv&sheet=Sheet1"
# orders_df = pd.read_csv(google_sheet_url)


# Sample data
orders_df = pd.DataFrame(
    {
        "unit_id": [101, 102, 103, 104, 105],
        "hex_x": [1, 3, 5, 7, 9],
        "hex_y": [4, 6, 8, 1, 3],
        "putative": [True, False, True, False, True],
    }
)

orders = []
for i, row in orders_df.iterrows:
    order_type = "putative" if row["putative"] else "regular"
    orders.append(
        Order(
            unit_id=row["unit_id"],
            hex_x=row["hex_x"],
            hex_y=row["hex_y"],
            order_type=order_type,
            map=game.map,
        )
    )




### Example

#### Entire turn

Show order lists being passed to run a turn


In [ ]:
from warshard.actions import Order

pending_orders = [
    Order(unit_id=1, hex_x=3, hex_y=4, map=game.map),
    Order(unit_id=1, hex_x=4, hex_y=4, map=game.map),
    # etc.
]

In [ ]:
game.run_a_turn(pending_orders)

Explain the principle of the orders.

Note : here the "attacker" designated the active player.

Here are the kind of orders that must be given each turn :

- Attacker must give movement orders for their units. To move a unit more than one hex, give successive movements orders (ie. a1 to a2, then a2 to a3)
- Attacker must also give its attack orders, for both melee and support units
- Defender must give its defensive orders for support units.
  - In the classical setup (using run_a_turn and not the individual turn phase functions), all orders must be provided at once.  If, in your setup, you allow the defender to see the attacker's order before providing their orders (and then group them all and only then pass them to run_a_turn), they can assign their support in reaction. However, if in your setup you want them to provide orders at once and without having seen the other's orders, the defender needs to anticipate and assign priority. For example, let's say the defender believes that the attacker may attack on hex H1 or hex H2 and has two artillery units. The defender may decide to give both artillery units an order to defend H1 which is to them more important, and then below in the list an order to defend H2. If the attacker attacks H1, the order will be executed. However if the attacker attacks H2 and not H1, the defend order for H1 will be invalid and thus ignored, and the lower-priority order for H2 will be executed. Finally, if the attacker attacks both H1 and H2, both orders are valid, but since the order to defend H1 is higher priority it will be executed and not the one for H2.
- Both attacker and defender must provide putative retreats and putative advances. Here too, there is a priority system : orders first in the list will be executed first and any subsequent order will be ignored, so you can for example specify two possible axis of retreat and depending on the outcome of other fights the first valid one will be used


#### Individual phases

Show order lists being passed to run individual phases.

the run_a_turn function does little more than run each of these in order. If you want more granularity, you can. Look at the source code.

HOWEVER this will break turn handling logic and will break memorization of order in the game.all_orders_ever_given

## Debug

Where to look : objects such as game.map give the info you need. You can use this to read the map/gamestate and even modify it, it is accessible

Also debug commands to give individual orders, spawn and move units, etc.
There are also functions that give individual orders (ie. start a fight here, etc.) so the simulation can be run with granulatiry in  a notebook, as if we were playing.  But in general you want to try to use run_a_turn as much as possible.

In [ ]:
game.map

## Developing an AI

Example of a recommended setup for AI development

In [ ]:
class StupidAI:
    """Simple example of a stupid AI"""

    def __call__(self, gamestate_representation) -> list[Order]:
        raise NotImplementedError
        for unit in self.units_i_am_responsible_for:
            closest_vp = ...  # Find the closest victory point to my position
            my_orders += [...]  # Issue orders in a straight line towards it
        return my_orders


my_custom_ai_agent = StupidAI()

In [ ]:
# Read the gamestate according to your desires and constraits (what do you allow the AI algorithm to know ? what representation does it need ?)
custom_gamestate_representation_observation = analyse_gamestate(game.map)


# Define a reward - here is a simple example
def reward_function(game):
    return game.map.hexgrid.get_total_victory_points_per_players()


# Now ask the AI to produce a list of pending orders
pending_orders = my_custom_ai_agent(custom_gamestate_representation_observation)

# Run the turn
game.run_a_turn(pending_orders)

# Evaluate the result
result_reward = reward_function(game)